In [1]:
import requests
import json
import chess.pgn
import io
import pandas as pd
import math
import numpy as np
import time

pd.set_option('display.max_rows', 50)

In [2]:
def get_data_by_month(username, year, month):

    url = f"https://api.chess.com/pub/player/{username}/games/{year}/{month}"

    data = requests.get(url)
    if data.status_code != 200:
        raise Exception("The following response was returned: " + str(data.status_code))
    else:
        data = json.loads(data.text)
        games = data["games"]
        
    all_games=[]
    for game in games:
        pgn = (game['pgn'])
        pgn = io.StringIO(pgn)
        game = chess.pgn.read_game(pgn)
        all_games.append(game)
                
    game_list = []
    for g in all_games:
        moves = (g.mainline_moves())
        moves = [str(x) for x in moves]
        
        white = (g.headers['White'])
        if white.lower() == username.lower():
            playing_as_white = 1
        else:
            playing_as_white = 0
        
        if len(moves)>1:
            move_made = (moves[1])
        else:
            move_made = ""
        
        game = {"date": (g.headers["Date"]), "player_white": white, "player_black": (g.headers['Black']), "playing_as_white" : playing_as_white, "result": (g.headers['Result']), "termination": (g.headers['Termination']), "moves": moves, "no_of_moves": (math.ceil(len(moves)/2)), "first_move": (moves[0]), "response": move_made}
    
        
        game_list.append(game)
    game_list = pd.DataFrame(game_list)
    return game_list


In [3]:
this_year = [("2021", "01"),  ("2021", "02"), ("2021", "03"), ("2021", "04"), ("2021", "05")]

In [4]:
all_months = []
for date in this_year:
    year = date[0]
    month = date[1]
    
    df = get_data_by_month("", year, month)
    all_months.append(df)
    time.sleep(10)
    print("Sleeping")

Sleeping
Sleeping
Sleeping
Sleeping
Sleeping


In [5]:
def combine_months(dfs):
    df = pd.concat(dfs, ignore_index=True)
    return df

In [6]:
all_months = combine_months(all_months)

In [8]:
def drop_not_required_columns(df):
    # For now I am not interested in these columns
    df = df.drop(["player_white", "player_black", "moves", "termination"], axis =1)
    return df

In [9]:
all_months = drop_not_required_columns(all_months)

In [11]:
 def create_wins_column(df):
    
    conditions = \
    [(df["playing_as_white"] == 1) & (df["result"] == "1-0"), 
     (df["playing_as_white"] == 1) & (df["result"] == "0-1"), 
     (df["playing_as_white"] == 0) & (df["result"] == "1-0"), 
     (df["playing_as_white"] == 0) & (df["result"] == "0-1"), 
     (df["playing_as_white"] == 1) & (df["result"] == "1/2-1/2"),
     (df["playing_as_white"] == 0) & (df["result"] == "1/2-1/2")]
    
    values = ["Win", "Loss", "Loss", "Win", "Draw", "Draw"]
                
    df['my_result'] = np.select(conditions, values)    

    return df    

In [12]:
all_months = create_wins_column(all_months)

In [14]:
def column_by_month(df):
    df["date"] = pd.to_datetime(df["date"])
    df["month"] = pd.DatetimeIndex(df["date"]).month    
    return df

In [15]:
all_months = column_by_month(all_months)

In [16]:
all_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              430 non-null    datetime64[ns]
 1   playing_as_white  430 non-null    int64         
 2   result            430 non-null    object        
 3   no_of_moves       430 non-null    int64         
 4   first_move        430 non-null    object        
 5   response          430 non-null    object        
 6   my_result         430 non-null    object        
 7   month             430 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 27.0+ KB


In [17]:
# see my most common opening
all_months[all_months["playing_as_white"]==1].groupby(["first_move", "my_result"])["my_result"].count()

first_move  my_result
c2c3        Loss           1
            Win            1
d2d4        Draw           8
            Loss          91
            Win          113
e2e4        Loss           2
            Win            2
Name: my_result, dtype: int64

In [18]:
#those of my opponents

In [19]:
pd.set_option('display.max_rows', None)
all_months[all_months["playing_as_white"]==0].groupby(["first_move", "my_result"])["my_result"].count()

first_move  my_result
a2a4        Loss          1
b1c3        Win           2
c2c4        Loss          3
d2d3        Loss          1
d2d4        Draw          1
            Loss         29
            Win          21
e2e3        Loss          1
            Win           2
e2e4        Draw          5
            Loss         65
            Win          69
f2f3        Draw          1
f2f4        Win           1
g1f3        Loss          4
            Win           1
g2g3        Loss          1
            Win           1
h2h4        Win           3
Name: my_result, dtype: int64

In [20]:
#zooming in on these openings and how I respond
all_months[(all_months["playing_as_white"]==0) & ((all_months["first_move"]=="e2e4") | (all_months["first_move"]=="d2d4"))].groupby(["first_move", "response", "my_result"])["my_result"].count()

first_move  response  my_result
d2d4        a7a5      Loss          2
            b8c6      Loss          3
                      Win           1
            c7c6      Loss          1
                      Win           1
            d7d5      Draw          1
                      Loss          7
                      Win          10
            e7e5      Loss          4
                      Win           5
            e7e6      Win           1
            g7g6      Loss          8
                      Win           2
            g8f6      Loss          4
            h7h5      Win           1
e2e4                  Loss          1
            a7a5      Loss          4
                      Win           1
            b8c6      Loss          1
            c7c5      Loss          1
                      Win           1
            c7c6      Loss          1
                      Win           1
            d7d5      Draw          1
                      Win           1
            d7d6  